<h2 style="font-family: 'Georgia'; color: #34495e; border-bottom: 3px solid #3498db; padding-bottom: 10px;">Importazione dei Pip necessari</h2>
<p style="font-family: 'Arial'; font-size: 16px; color: #2c3e50; margin-top: 20px;">Vengono importati i pip necessari per fare funzionare lo script</p>
<ul style="color: #34495e; font-size: 16px;">
<li>nel caso non siano già installati, usare il commento nel terminale</li>
</ul>
<ul style="color: #34495e; font-size: 16px;">
<li>viene aggiunto un breve CSS per poter produrre testi e output in maniera coerente</li>
</ul>

In [34]:
#pip3 install chardet pandas folium  bs4 tqdm numpy matplotlib cartopy display Markdown Latex
import chardet  
import pandas as pd
import folium
import os
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm
import zipfile
import numpy as np
import cartopy.feature as cfeature
from IPython.display import display, HTML
import markdown

# Custom CSS
custom_css = """
<style>

    h2{
        font-family: 'Georgia';
        color: #34495e;
        border-bottom: 3px solid #3498db;
        padding-bottom: 10px;
        
    }
        
    p {
        font-family: 'Arial';
        font-size: 16px;
        color: #2c3e50;
        margin-top: 20px;
        
    }
    
    ul{
    
        color: #34495e;
        font-size: 16px;
    
    }
    
</style>
"""

## Determinare i dataset utili
Vista la grande quantità di dati con cui si sta lavorando, è opportuno scaricare i dataset strettamente necessari. La seguente porzione di codice fa riferimento al file <a href='https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/KL_Tageswerte_Beschreibung_Stationen.txt' >"KL_Tageswerte_Beschreibung_Stationen.txt"</a> che ci consente di determinare le stazioni meteo attive nel periodo a noi di interesse.

Per qualche ragione il file non è codificato in UTF-8, rendendo quindi necessaria determinazione del il tipo di codifica usato per poter aprire il dataset (che poi provvederemo a codificare in UTF-8 per renderne più agevole l'accesso).

Una volta determinate le stazioni meteo a noi necessarie, creiamo un dataset con solo esse presenti.

In [46]:
derectory='datasets/KL_Tageswerte_Beschreibung_Stationen.txt'
with open(derectory, 'rb') as f:
    enc = chardet.detect(f.read(10000))
    html_text = markdown.markdown(f"**Codifica rilevata:** `{enc['encoding']}`")
    final_output = custom_css + html_text
display(HTML(final_output))


columns = [    "Stations_id", "da_data", "fino_a_data",
    "Stationshoehe", "latitude", "longitude",
    "Stationsname", "Bundesland", "Abgabe"
]

dataset = pd.read_fwf(derectory, encoding=enc['encoding'],names=columns)

dataset['da_data'] = pd.to_datetime(dataset['da_data'], format='%Y%m%d', errors='coerce')
dataset['fino_a_data'] = pd.to_datetime(dataset['fino_a_data'], format='%Y%m%d', errors='coerce')

dataset

,Stations_id,da_data,fino_a_data,Stationshoehe,latitude,longitude,Stationsname,Bundesland,Abgabe
0,1,1937-01-01,1986-06-30,478,47.8413,8.8493,Aach,Baden-Württemberg,Frei
1,3,1891-01-01,2011-03-31,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen,Frei
2,11,1980-09-01,2025-05-29,680,47.9736,8.5205,Donaueschingen (Landeplatz),Baden-Württemberg,Frei
3,44,1969-01-01,2025-05-29,44,52.9336,8.2370,Großenkneten,Niedersachsen,Frei
4,52,1969-01-01,2001-12-31,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schleswig-Holstein,Frei
...,...,...,...,...,...,...,...,...,...
1372,19911,1963-11-01,1986-09-30,215,50.9625,10.0663,Gerstungen/Thüringen,Thüringen,Frei
1373,19985,1981-01-01,2003-12-31,455,48.8457,8.9073,Weissach/Strudelbach,Baden-Württemberg,Frei
1374,19989,1936-01-01,2012-06-30,147,50.0681,8.2603,Wiesbaden (Südost),Hessen,Frei
1375,20007,1882-01-01,1915-12-31,332,50.8422,12.8770,Chemnitz-Altendorf,Sachsen,Frei


## Rappresentazione delle stazioni meteo
Per comprendere meglio la disposizione delle stazioni meteo, vengono rappresentate tre mappe, la prima con le stazioni totali, la seconda dove vengono segnalate le stazioni non attive nel periodo selezionato, nella terza solo le stazioni attive nel periodo

In [36]:
# Determine unique latitude-longitude pairs
unique_locations = dataset[['latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
html_text = markdown.markdown(f"Sono presenti {unique_locations.shape[0]} colonnine meteo totali.")
final_output = custom_css + html_text
display(HTML(final_output))


## Visualizzazione grafica
Per potere visualizzare la posizione delle stazioni meteo, uno breve script ci consente di geolocalizzarle all'interno di una mappa e salvarla nel caso si volesse analizzarla più approfonditamente in un secondo momento.

In [43]:
# Crea una mappa centrata approssimativamente sulla Germania
m = folium.Map(location=[46.0710, 13.2345], zoom_start=5, tiles="OpenStreetMap", width=800, height=800)

# Aggiungi ogni stazione come marker
for idx, row in dataset.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    nome = row['Stationsname']
    folium.CircleMarker(
        location=[lat, lon],
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)

m

In [44]:
m.save("per_presentazione/confronto_mappe/mappa_stazioni_attive_totali.html")
html_text=markdown.markdown('mappa salvata in: per_presentazione/confronto_mappe/mappa_stazioni_attive_totali.html')
final_output = custom_css + html_text
display(HTML(final_output))

## Determinare un periodo temporale
In questa circostanza, prenderemo come periodo temporale per l'analisi la durata dell'anno 2020, a seguito uno script per determinare le stazioni meteo attive nel dato periodo.

Il CSV ricavato viene salvato per essere ulizzato sucessivamente.

In [45]:
start = pd.to_datetime("2020-01-01")
end = pd.to_datetime("2020-12-31")
active_nel_periodo = dataset[
    (dataset['da_data'] <= end) & 
    (dataset['fino_a_data'] >= start)
]


active_nel_periodo.to_csv('elaborazione_dati/per_scraping/attivo_nel_2020.csv', index=False)
html_text=markdown.markdown('CSV salvato in: elaborazione_dati/per_scraping/attivo_nel_2020.csv')
final_output = custom_css + html_text
display(HTML(final_output))

## Confronto colonnine meteo
Vengono evidenziate ora in rosso le stazioni meteo che non sono attive nel periodo selezionato (anno 2020), nelle analisi sucessive verranno escluse per risparmiare dati e rendere l'analisi più semplice.

In [29]:
# Crea una mappa centrata approssimativamente sulla Germania
m = folium.Map(location=[46.0710, 13.2345], zoom_start=5, tiles="OpenStreetMap", width=800, height=800)

# Aggiungi ogni stazione come marker
for idx, row in dataset.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    nome = row['Stationsname']
    active_nel_periodo = (row['da_data'] <= end) and (row['fino_a_data'] >= start)
    color = 'blue' if active_nel_periodo else 'red',
    folium.CircleMarker(
        location=[lat, lon],
        radius=2,
        fill=True,
        color=color,
        fill_color=color,
        fill_opacity=0.7
    ).add_to(m)

m

In [47]:
m.save("per_presentazione/confronto_mappe/mappa__differenza_stazioni_attive_nel_2020.html")
html_text=markdown.markdown('Mappa salvata in: per_presentazione/confronto_mappe/mappa__differenza_stazioni_attive_nel_2020.html')
final_output = custom_css + html_text
display(HTML(final_output))

## Stazioni meteo selezionate
A seguito, la locazione delle stazioni meteo in germania attive nel 2020.

In [10]:
# Crea una mappa centrata approssimativamente sulla Germania
m = folium.Map(location=[46.0710, 13.2345], zoom_start=5, tiles="OpenStreetMap", width=800, height=800)


# Aggiungi ogni stazione come marker
for idx, row in df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    nome = row['Stations_id']
    folium.CircleMarker(
        location=[lat, lon],
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)

m

In [32]:
m.save("per_presentazione/confronto_mappe/mappa_stazioni_attive_nel_2020.html")
html_text=markdown.markdown('Mappa salvata in: per_presentazione/confronto_mappe/mappa_stazioni_attive_nel_2020.html')
final_output = custom_css + html_text
display(HTML(final_output))
html_text=markdown.markdown('Dataset aggiornato salvato')
final_output = custom_css + html_text
display(HTML(final_output))

In [29]:
# Determine unique latitude-longitude pairs
unique_locations = df[['latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
#print(f"Sono presenti {df.shape[0]} colonnine meteo attive nel 2020.")
html_text = markdown.markdown(f"**Sono presenti:** {df.shape[0]} colonnine meteo attive nel 2020.")
final_output = custom_css + html_text
display(HTML(final_output))

## Il nuovo dataset "pulito"
Dopo l'opportuna elaborazione, rispetto al dataset iniziale sono presenti circa 800 stazioni meteo in meno che non sarebbero state utilizzate nell'analisi dei dati, in quanto attive in un periodo temporale diverso da quello analizzato.

In [30]:
dataset = pd.read_csv('elaborazione_dati/per_scraping/attivo_nel_2020.csv')

dataset

,Stations_id,da_data,fino_a_data,Stationshoehe,latitude,longitude,Stationsname,Bundesland,Abgabe
0,11,1980-09-01,2025-05-29,680,47.9736,8.5205,Donaueschingen (Landeplatz),Baden-Württemberg,Frei
1,44,1969-01-01,2025-05-29,44,52.9336,8.2370,Großenkneten,Niedersachsen,Frei
2,73,1959-03-01,2025-05-29,374,48.6183,13.0620,Aldersbach-Kramersepp,Bayern,Frei
3,78,1961-01-01,2025-05-29,64,52.4853,7.9125,Alfhausen,Niedersachsen,Frei
4,90,1988-02-19,2025-05-29,305,50.7557,9.2583,Alsfeld,Hessen,Frei
...,...,...,...,...,...,...,...,...,...
570,15801,2020-10-01,2025-05-29,718,48.8257,13.5922,Hinterschmiding-Sonndorf,Bayern,Frei
571,15976,2020-04-02,2025-05-29,216,49.9529,8.2107,Ober-Olm/Bellem,Rheinland-Pfalz,Frei
572,15978,2020-04-02,2025-05-29,316,49.5044,7.8618,Sembach,Rheinland-Pfalz,Frei
573,19171,2020-09-01,2025-05-29,13,54.0039,9.8555,Hasenkrug-Hardebek,Schleswig-Holstein,Frei


In [15]:
# === CONFIGURAZIONE ===
BASE_URL = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/"
CSV_PATH = "per_scraping/attivo_nel_2020.csv"  # percorso del tuo file CSV
DOWNLOAD_DIR = "datasets/COLONNINE_METEO/dati_stazioni_attive_2020"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# === STEP 1: Leggi gli ID delle stazioni dal CSV ===
station_ids = set()
with open(CSV_PATH, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # supponiamo che l'id sia nella colonna chiamata "STATIONS_ID"
        # puoi fare print(row.keys()) per verificare il nome esatto
        station_id = row.get("STATIONS_ID") or row.get("Stations_id") or row.get("station_id")
        if station_id:
            station_ids.add(station_id.zfill(5))  # formatta l'ID con zeri davanti (es: "1048" -> "01048")

print(f"Loaded {len(station_ids)} station IDs")

# === STEP 2: Scarica e analizza la pagina HTML ===
response = requests.get(BASE_URL)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

# === STEP 3: Filtra i file ZIP che contengono un ID valido ===
for link in tqdm(soup.find_all("a"),'download datasets'):
    href = link.get("href")
    if href and href.endswith(".zip"):
        for station_id in station_ids:
            if station_id in href:
                file_url = BASE_URL + href
                local_path = os.path.join(DOWNLOAD_DIR, href)

                print(f"Downloading: {file_url}")
                file_response = requests.get(file_url)
                file_response.raise_for_status()
                with open(local_path, "wb") as f:
                    f.write(file_response.content)
                print(f"Saved to: {local_path}")
                break  # se abbiamo trovato un ID valido, saltiamo al prossimo link

Loaded 575 station IDs


download datasets:   0%|                                                                      | 0/1283 [00:00<?, ?it/s]

Downloading: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/tageswerte_KL_00003_18910101_20110331_hist.zip


download datasets:   0%|▏                                                             | 4/1283 [00:01<08:23,  2.54it/s]

Saved to: datasets/COLONNINE_METEO/dati_stazioni_attive_2020\tageswerte_KL_00003_18910101_20110331_hist.zip
Downloading: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/tageswerte_KL_00011_19800901_20241231_hist.zip


...

download datasets:  98%|█████████████████████████████████████████████████████████▋ | 1255/1283 [03:35<00:02, 13.98it/s]

Saved to: datasets/COLONNINE_METEO/dati_stazioni_attive_2020\tageswerte_KL_16085_19600701_19611231_hist.zip
Downloading: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/tageswerte_KL_19171_20200901_20241231_hist.zip


download datasets:  98%|█████████████████████████████████████████████████████████▊ | 1257/1283 [03:35<00:02,  9.58it/s]

Saved to: datasets/COLONNINE_METEO/dati_stazioni_attive_2020\tageswerte_KL_19171_20200901_20241231_hist.zip
Downloading: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/tageswerte_KL_19172_20200901_20241231_hist.zip
Saved to: datasets/COLONNINE_METEO/dati_stazioni_attive_2020\tageswerte_KL_19172_20200901_20241231_hist.zip


download datasets: 100%|███████████████████████████████████████████████████████████| 1283/1283 [03:35<00:00,  5.94it/s]


In [16]:
# Input and output folders
zip_dir = "datasets/COLONNINE_METEO/dati_stazioni_attive_2020"
output_dir = "datasets/COLONNINE_METEO/stazioni_ordinate"
os.makedirs(output_dir, exist_ok=True)

# Get all .zip files
zip_files = [
    f for f in os.listdir(zip_dir)
    if f.endswith(".zip") and f.startswith("tageswerte_KL_")
]

# Loop over all zip files with progress bar
for zip_filename in tqdm(zip_files, desc="Extracting files"):
    zip_path = os.path.join(zip_dir, zip_filename)

    #individua l'id
    ID= zip_filename[14:19]
    
    # Remove _hist.zip extension
    base_name = zip_filename[:37]

    # Remove 'tageswerte_KL_' prefix
    name_parts = base_name[20:]
    

    # Construct expected file name inside the zip
    expected_file = f"produkt_klima_tag_{name_parts}_{ID}.txt"

    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Search for the file (even if it's in a subfolder)
            matched_file = next(
                (f for f in zip_ref.namelist() if os.path.basename(f) == expected_file),
                None
            )

            if matched_file:
                zip_ref.extract(matched_file, output_dir)
            else:
                tqdm.write(f"❌ Not found in {zip_filename}: {expected_file} {ID}")
    except zipfile.BadZipFile:
        tqdm.write(f"❌ Corrupted zip file: {zip_filename}")


Extracting files: 100%|██████████████████████████████████████████████████████████████| 597/597 [00:28<00:00, 20.95it/s]


In [17]:
#I commenti sono la trascrizione degli elementi più rilevanti di DESCRIPTION_obsgermany-climate-daily-kl_en.pdf
#Per maggiori dettagli consultare il file

     #    column name       #  description                                              uom           type             format
columns=["STATIONS_ID",     #  Station ID                                               ---           VARCHAR2         ------
         "MESS_DATUM",      #  reference date                                           ---           NUMBER           YYYYMMDD
         "QN_3",            #  quality level of the following columns                   ---           NUMBER           numerical code
         "FX",              #  daily maximum of windgust                                m/s           NUMBER           9990.0
         "FM",              #  daily mean of wind velocity                              m/s           NUMBER           9990.0
         "QN_4",            #  quality level of the following columns                   ---           NUMBER           numerical code
         "RSK",             #  daily precipitation height                               mm            NUMBER           9990.0
         "RSKF",            #  precipitation form                                       num.code      NUMBER           ---
         "SDK",             #  daily sunshine duration                                  h             NUMBER           9990.0
         "SHK_TAG",         #  daily snow depth                                         cm            NUMBER           9990.0
         "NM",              #  daily mean of cloud cover                                1/8           NUMBER           9990.0
         "VPM",             #  daily mean of vapor pressure                             hPa           NUMBER           9990.0
         "PM",              #  daily mean of pressure                                   hPa           NUMBER           9990.0
         "TMK",             #  daily mean of temperature                                °C            NUMBER           9990.0
         "UPM",             #  daily mean of relative humidity                          %             NUMBER           9990.0
         "TXK",             #  daily maximum of temperature at 2 m height               °C            NUMBER           9990.0
         "TNK",             #  daily minimum of temperature at 2m heigh                 °C            NUMBER           9990.0
         "TGK",             # daily minimum of air temperature at 5 cm above ground     °C            NUMBER           9990.0
         "eor"]             #

In [18]:
# Set the directory path
folder_path = 'datasets/COLONNINE_METEO/stazioni_ordinate'  # e.g., 'C:/Users/You/Documents/data'

# List all .txt files in the folder
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# Read each .txt file as a CSV into a list of DataFrames
dfs = []
for file in txt_files:
    file_path = os.path.join(folder_path, file)
    
    # You can specify sep="," or sep="\t" based on your file's format
    df = pd.read_csv(file_path, sep=";", header=0)  
    dfs.append(df)

# Example: concatenate all dataframes together
combined_df = pd.concat(dfs, ignore_index=True)

# Optional: print or save
combined_df
combined_df.to_csv('datasets/COLONNINE_METEO/combined_output.csv', index=False)

#puliamo il dataset dai periodi temporali non interessati

df= pd.read_csv('datasets/COLONNINE_METEO/combined_output.csv')
df['MESS_DATUM'] = pd.to_datetime(df['MESS_DATUM'], format='%Y%m%d', errors='coerce')
df

,STATIONS_ID,MESS_DATUM,QN_3,FX,FM,QN_4,RSK,RSKF,SDK,SHK_TAG,NM,VPM,PM,TMK,UPM,TXK,TNK,TGK,eor
0,2290,1781-01-01,-999,-999.0,-999.0,8,-999.0,-999,-999.0,-999,-999.0,-999.0,899.1,3.3,-999.0,-999.0,-999.0,-999.0,eor
1,2290,1781-01-02,-999,-999.0,-999.0,8,-999.0,-999,-999.0,-999,-999.0,-999.0,891.8,0.3,-999.0,-999.0,-999.0,-999.0,eor
2,2290,1781-01-03,-999,-999.0,-999.0,8,-999.0,-999,-999.0,-999,-999.0,-999.0,890.4,-4.7,-999.0,-999.0,-999.0,-999.0,eor
3,2290,1781-01-04,-999,-999.0,-999.0,8,-999.0,-999,-999.0,-999,-999.0,-999.0,896.0,-6.7,-999.0,-999.0,-999.0,-999.0,eor
4,2290,1781-01-05,-999,-999.0,-999.0,8,-999.0,-999,-999.0,-999,-999.0,-999.0,901.9,-8.8,-999.0,-999.0,-999.0,-999.0,eor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11054854,15801,2024-12-27,10,9.8,3.4,-999,-999.0,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,eor
11054855,15801,2024-12-28,10,3.7,1.8,-999,-999.0,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,eor
11054856,15801,2024-12-29,10,3.3,1.4,-999,-999.0,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,eor
11054857,15801,2024-12-30,10,2.7,1.1,-999,-999.0,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,eor


In [20]:
start = pd.to_datetime("2020-01-01")
end = pd.to_datetime("2020-12-31")
active_nel_periodo = df[(df['MESS_DATUM'] <= end) & (df['MESS_DATUM'] >= start)]


active_nel_periodo.to_csv('datasets/COLONNINE_METEO/combined_output_active_in_period.csv', index=False)
active_nel_periodo

,STATIONS_ID,MESS_DATUM,QN_3,FX,FM,QN_4,RSK,RSKF,SDK,SHK_TAG,NM,VPM,PM,TMK,UPM,TXK,TNK,TGK,eor
85450,2290,2020-01-01,10,7.1,3.4,9,0.0,0,7.9,0,0.0,4.5,915.6,2.8,60.0,6.6,-0.2,-2.3,eor
85451,2290,2020-01-02,10,7.8,2.4,9,0.0,0,7.9,0,0.5,3.7,913.0,5.0,43.0,8.5,2.4,-0.6,eor
85452,2290,2020-01-03,10,18.8,6.5,9,1.3,8,3.0,0,6.8,5.2,910.9,4.1,64.0,7.2,0.7,-0.6,eor
85453,2290,2020-01-04,10,16.5,7.0,9,2.6,8,0.0,0,7.7,6.4,913.7,1.0,97.0,2.8,-0.8,-0.9,eor
85454,2290,2020-01-05,10,14.3,3.8,9,0.0,8,2.0,1,6.3,5.5,917.9,-0.3,92.0,1.6,-1.6,-2.6,eor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11053393,15801,2020-12-27,10,8.5,2.4,-999,-999.0,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,eor
11053394,15801,2020-12-28,10,8.5,2.5,-999,-999.0,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,eor
11053395,15801,2020-12-29,10,7.3,1.9,-999,-999.0,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,eor
11053396,15801,2020-12-30,10,3.7,1.6,-999,-999.0,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,eor


In [21]:
df_pulito = active_nel_periodo.replace(-999.0, np.nan)

df_pulito.to_csv('datasets/COLONNINE_METEO/df_pulito.csv', index=False)
df_pulito

,STATIONS_ID,MESS_DATUM,QN_3,FX,FM,QN_4,RSK,RSKF,SDK,SHK_TAG,NM,VPM,PM,TMK,UPM,TXK,TNK,TGK,eor
85450,2290,2020-01-01,10.0,7.1,3.4,9.0,0.0,0.0,7.9,0.0,0.0,4.5,915.6,2.8,60.0,6.6,-0.2,-2.3,eor
85451,2290,2020-01-02,10.0,7.8,2.4,9.0,0.0,0.0,7.9,0.0,0.5,3.7,913.0,5.0,43.0,8.5,2.4,-0.6,eor
85452,2290,2020-01-03,10.0,18.8,6.5,9.0,1.3,8.0,3.0,0.0,6.8,5.2,910.9,4.1,64.0,7.2,0.7,-0.6,eor
85453,2290,2020-01-04,10.0,16.5,7.0,9.0,2.6,8.0,0.0,0.0,7.7,6.4,913.7,1.0,97.0,2.8,-0.8,-0.9,eor
85454,2290,2020-01-05,10.0,14.3,3.8,9.0,0.0,8.0,2.0,1.0,6.3,5.5,917.9,-0.3,92.0,1.6,-1.6,-2.6,eor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11053393,15801,2020-12-27,10.0,8.5,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor
11053394,15801,2020-12-28,10.0,8.5,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor
11053395,15801,2020-12-29,10.0,7.3,1.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor
11053396,15801,2020-12-30,10.0,3.7,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor


In [22]:
print(df_pulito.describe(include='all'))

          STATIONS_ID                     MESS_DATUM      QN_3             FX  \
count   204292.000000                         204292  103611.0  102552.000000   
unique            NaN                            NaN       NaN            NaN   
top               NaN                            NaN       NaN            NaN   
freq              NaN                            NaN       NaN            NaN   
mean      4144.746084  2020-07-01 18:33:28.043388416      10.0      10.655892   
min         11.000000            2020-01-01 00:00:00      10.0       1.500000   
25%       1721.000000            2020-04-01 00:00:00      10.0       7.600000   
50%       3545.000000            2020-07-02 00:00:00      10.0       9.800000   
75%       5676.000000            2020-10-01 00:00:00      10.0      12.900000   
max      19172.000000            2020-12-31 00:00:00      10.0      49.100000   
std       3311.581862                            NaN       0.0       4.457660   

                   FM      

In [23]:
input_dir = 'datasets/DATASET_NASA_2020/'
output_dir = 'elaborazione_dati/csv_output/'

os.makedirs(output_dir, exist_ok=True)
files = [f for f in os.listdir(input_dir) if f.endswith('.h5')]
DATAFIELD_NAME = 'Soil_Moisture_Retrieval_Data_AM/soil_moisture'

print(f"Trovati {len(files)} file .h5 da processare.")

for file_name in tqdm(files, desc="Processando file HDF5"):
    input_path = os.path.join(input_dir, file_name)
    print(f"Processando file: {file_name}")

    # Apro il file HDF5
    with h5py.File(input_path, mode='r') as f:
        dset = f[DATAFIELD_NAME]
        data = dset[:]

        lat = f['Soil_Moisture_Retrieval_Data_AM/latitude'][:]
        lon = f['Soil_Moisture_Retrieval_Data_AM/longitude'][:]

        # Gestisco fillvalue
        fillvalue = dset.attrs['_FillValue']
        data[data == fillvalue] = np.nan

        # Flatten per creare il DataFrame
        lat_flat = lat.flatten()
        lon_flat = lon.flatten()
        data_flat = data.flatten()

        df = pd.DataFrame({
            'latitude': lat_flat,
            'longitude': lon_flat,
            'soil_moisture': data_flat
        })

        # Rimuovo righe con soil_moisture NaN
        df = df.dropna(subset=['soil_moisture'])

        # Creo il nome del CSV in base al nome del file HDF5
        # Es: SMAP_L3_SM_P_E_20200101_R19240_001.h5 → soil_moisture_20200101.csv
        base_name = os.path.splitext(file_name)[0]
        # Estraggo la data se c'è nel nome file (20200101)
        date_str = base_name.split('_')[5] if len(base_name.split('_')) > 5 else 'unknown_date'

        csv_name = f'soil_moisture_{date_str}.csv'
        csv_path = os.path.join(output_dir, csv_name)

        # Salvo il CSV
        df.to_csv(csv_path, index=False, float_format='%.5f')
        print(f"→ CSV salvato: {csv_path} ({len(df)} punti validi)")

print("Processamento completo!")

Trovati 366 file .h5 da processare.


Processando file HDF5:   0%|                                                                   | 0/366 [00:00<?, ?it/s]

Processando file: SMAP_L3_SM_P_E_20200101_R19240_001.h5


Processando file HDF5:   0%|▏                                                          | 1/366 [00:03<22:02,  3.62s/it]

→ CSV salvato: elaborazione_dati/csv_output/soil_moisture_20200101.csv (498171 punti validi)
Processando file: SMAP_L3_SM_P_E_20200102_R19240_001.h5


...

→ CSV salvato: elaborazione_dati/csv_output/soil_moisture_20201229.csv (464276 punti validi)
Processando file: SMAP_L3_SM_P_E_20201230_R19240_001.h5


Processando file HDF5: 100%|████████████████████████████████████████████████████████▊| 365/366 [26:01<00:03,  3.20s/it]

→ CSV salvato: elaborazione_dati/csv_output/soil_moisture_20201230.csv (428944 punti validi)
Processando file: SMAP_L3_SM_P_E_20201231_R19240_001.h5


Processando file HDF5: 100%|█████████████████████████████████████████████████████████| 366/366 [26:05<00:00,  4.28s/it]

→ CSV salvato: elaborazione_dati/csv_output/soil_moisture_20201231.csv (467235 punti validi)
Processamento completo!


In [24]:
input_dir = 'datasets/DATASET_NASA_2020/'
output_dir = 'elaborazione_dati/csv_output_pm/'

os.makedirs(output_dir, exist_ok=True)
files = [f for f in os.listdir(input_dir) if f.endswith('.h5')]
DATAFIELD_NAME = '/Soil_Moisture_Retrieval_Data_PM/soil_moisture_pm'

print(f"Trovati {len(files)} file .h5 da processare.")

for file_name in tqdm(files, desc="Processando file HDF5"):
    input_path = os.path.join(input_dir, file_name)
    print(f"Processando file: {file_name}")

    # Apro il file HDF5
    with h5py.File(input_path, mode='r') as f:
        dset = f[DATAFIELD_NAME]
        data = dset[:]

        lat = f['Soil_Moisture_Retrieval_Data_AM/latitude'][:]
        lon = f['Soil_Moisture_Retrieval_Data_AM/longitude'][:]

        # Gestisco fillvalue
        fillvalue = dset.attrs['_FillValue']
        data[data == fillvalue] = np.nan

        # Flatten per creare il DataFrame
        lat_flat = lat.flatten()
        lon_flat = lon.flatten()
        data_flat = data.flatten()

        df = pd.DataFrame({
            'latitude': lat_flat,
            'longitude': lon_flat,
            'soil_moisture': data_flat
        })

        # Rimuovo righe con soil_moisture NaN
        df = df.dropna(subset=['soil_moisture'])

        # Creo il nome del CSV in base al nome del file HDF5
        # Es: SMAP_L3_SM_P_E_20200101_R19240_001.h5 → soil_moisture_20200101.csv
        base_name = os.path.splitext(file_name)[0]
        # Estraggo la data se c'è nel nome file (20200101)
        date_str = base_name.split('_')[5] if len(base_name.split('_')) > 5 else 'unknown_date'

        csv_name = f'soil_moisture_{date_str}_pm.csv'
        csv_path = os.path.join(output_dir, csv_name)

        # Salvo il CSV
        df.to_csv(csv_path, index=False, float_format='%.5f')
        print(f"→ CSV salvato: {csv_path} ({len(df)} punti validi)")

print("Processamento completo!")

Trovati 366 file .h5 da processare.


Processando file HDF5:   0%|                                                                   | 0/366 [00:00<?, ?it/s]

Processando file: SMAP_L3_SM_P_E_20200101_R19240_001.h5


Processando file HDF5:   0%|▏                                                          | 1/366 [00:03<19:11,  3.15s/it]

→ CSV salvato: elaborazione_dati/csv_output_pm/soil_moisture_20200101_pm.csv (429238 punti validi)
Processando file: SMAP_L3_SM_P_E_20200102_R19240_001.h5


...

→ CSV salvato: elaborazione_dati/csv_output_pm/soil_moisture_20201229_pm.csv (423000 punti validi)
Processando file: SMAP_L3_SM_P_E_20201230_R19240_001.h5


Processando file HDF5: 100%|████████████████████████████████████████████████████████▊| 365/366 [27:23<00:03,  3.58s/it]

→ CSV salvato: elaborazione_dati/csv_output_pm/soil_moisture_20201230_pm.csv (465989 punti validi)
Processando file: SMAP_L3_SM_P_E_20201231_R19240_001.h5


Processando file HDF5: 100%|█████████████████████████████████████████████████████████| 366/366 [27:26<00:00,  4.50s/it]

→ CSV salvato: elaborazione_dati/csv_output_pm/soil_moisture_20201231_pm.csv (443099 punti validi)
Processamento completo!


In [25]:
import os
import pandas as pd
from tqdm import tqdm

dir1 = 'elaborazione_dati/csv_output/'
dir2 = 'elaborazione_dati/csv_output_pm/'
output_dir = 'elaborazione_dati/merged_output/'

# Creo output dir se non esiste
os.makedirs(output_dir, exist_ok=True)

# Lista dei file nella cartella 1
files1 = [f for f in os.listdir(dir1) if f.endswith('.csv')]

print(f"Trovati {len(files1)} file in {dir1}")

# Loop con progress bar
for file1 in tqdm(files1, desc="Merging datasets"):
    # Percorso completo per file in dir1
    path1 = os.path.join(dir1, file1)

    # Cerco file corrispondente in dir2 (con lo stesso nome)
    file2 = file1.replace('.csv', '_pm.csv')
    path2 = os.path.join(dir2, file2)

    if not os.path.exists(path2):
        print(f"ATTENZIONE: file mancante in {dir2}: {file1} — salto")
        continue

    # Carico entrambi i CSV
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path2)

    # Rinomino colonne per evitare conflitto
    df1 = df1.rename(columns={'soil_moisture': 'soil1'})
    df2 = df2.rename(columns={'soil_moisture': 'soil2'})

    # Merge su latitude + longitude (outer per non perdere punti)
    df_merged = pd.merge(df1, df2, on=['latitude', 'longitude'], how='outer')

    # Calcolo media
    df_merged['soil_moisture'] = df_merged[['soil1', 'soil2']].mean(axis=1, skipna=True)

    # Salvo CSV unito
    out_path = os.path.join(output_dir, file1)
    df_merged[['latitude', 'longitude', 'soil_moisture']].to_csv(out_path, index=False, float_format='%.5f')

print("Merge completo! CSV salvati in merged_output/")

Trovati 366 file in elaborazione_dati/csv_output/


Merging datasets: 100%|██████████████████████████████████████████████████████████████| 366/366 [45:51<00:00,  7.52s/it]

Merge completo! CSV salvati in merged_output/


In [26]:
# Define the folder path
data_folder = 'elaborazione_dati/merged_output'  # replace with your folder
output_folder = 'per_presentazione/output_maps'
os.makedirs(output_folder, exist_ok=True)


# List CSV files (you can adapt the extension if needed)
file_list = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# Loop through each file

for filename in tqdm(file_list, desc="plotting maps (this might take a while)"):
    # Full path to file
    file_path = os.path.join(data_folder, filename)
    
    # Load the data
    df = pd.read_csv(file_path)
    
    # Example assumption: your CSV has 'longitude' and 'latitude' columns
    lon = df['longitude']
    lat = df['latitude']
    soil_moisture = df['soil_moisture'].values
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})

    
    # Add basic map features
    ax.coastlines()
    gl = ax.gridlines(draw_labels=True, linestyle='--', color='gray')
    gl.top_labels = False
    gl.right_labels = False
    
    # Plot the points (you can adapt marker style)
    sc = ax.scatter(lon, lat, c=soil_moisture, cmap='viridis_r', vmin=0, vmax=1, s=5, transform=ccrs.PlateCarree())
    
    # Title = filename (without extension)
    plt.title(f'Map for {os.path.splitext(filename)[0]}')
    
    # Save the figure
    output_filename = os.path.join(output_folder, f"map_{os.path.splitext(filename)[0]}.png")
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    
    # Close figure to avoid memory problems
    plt.close(fig)
    


plotting maps (this might take a while): 100%|█████████████████████████████████████| 366/366 [4:20:54<00:00, 42.77s/it]
